<a href="https://colab.research.google.com/github/mdereli5797/textepo/blob/main/Generalized_UMApHMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# Set the random seed for reproducibility
np.random.seed(42)
# Define the number of nodes (n)
n = 25  # You can adjust this number as needed

# Randomly generate integer x and y coordinates for nodes
x_coords = np.random.randint(0, 101, n)
y_coords = np.random.randint(0, 101, n)

# Generate F_hub vector (fixed cost of opening hubs) as integers
F_hub = np.random.randint(200000, 1200001, n)

# Generate W matrix (flow from node i to node j) with integer values between 100 and 500
W_matrix = np.random.randint(10, 201, (n, n))
W_matrix = (W_matrix + W_matrix.T) // 2  # Make the matrix symmetric
np.fill_diagonal(W_matrix, 0)  # Set diagonal to 0
np.fill_diagonal(W_matrix, 0)  # Set diagonal to 0

# Generate F_assignment matrix (assignment cost) with integer values between 0 and 5000
F_assignment_matrix = np.random.randint(5000, 400001, (n, n))
np.fill_diagonal(F_assignment_matrix, 0)  # Set diagonal to 0

# Create a Pandas DataFrame for x and y coordinates and F_hub
df_nodes = pd.DataFrame({
    'x': x_coords,
    'y': y_coords,
    'F_hub': F_hub
})

# Convert W and F_assignment to DataFrames for better visualization
df_W = pd.DataFrame(W_matrix, columns=[i for i in range(n)], index=[i for i in range(n)])
df_F_assignment = pd.DataFrame(F_assignment_matrix, columns=[i for i in range(n)], index=[i for i in range(n)])

# Display the generated data
#print("Nodes DataFrame (x, y, F_hub):")
#display(df_nodes)
#print("\nW Matrix (Flow Costs):")
#display(df_W)
#print("\nF Assignment Matrix (Assignment Costs):")
#display(df_F_assignment)


In [2]:
from scipy.spatial.distance import cdist

# Compute the distance matrix using Euclidean distance
coordinates = df_nodes[['x', 'y']].values  # Extract x and y coordinates as a NumPy array
distance_matrix = cdist(coordinates, coordinates, metric='euclidean')

# Convert the distance matrix into a Pandas DataFrame for better readability
df_distance = pd.DataFrame(
    distance_matrix,
    columns=[i for i in range(len(df_nodes))],
    index=[i for i in range(len(df_nodes))]
)

# Display the distance matrix
#print("Distance Matrix (Euclidean):")
#display(df_distance)


In [3]:
# Calculate the total weight for each node (sum of rows in W_matrix)
sum_W = W_matrix.sum(axis=1)

# Convert sum_W to a Pandas Series for better readability
sum_W_series = pd.Series(sum_W, index=[i for i in range(len(sum_W))], name='sum_W')

# Display the sum_W vector
#print("Total Weight (sum_W) for each node:")
#display(sum_W_series)


In [4]:
# Calculate the probability vector
total_sum_W = sum_W_series.sum()
probability_vector = sum_W_series / total_sum_W

# Display the probability vector
#print("Probability Vector:")
#display(probability_vector)
# Assert that the sum is approximately 1
assert np.isclose(probability_vector.sum(), 1), "The sum of the probability vector is not 1."
print("The sum of the probability vector is approximately 1.")


The sum of the probability vector is approximately 1.


In [5]:
# Define the parameter p for the number of hubs
p = 8

# Print the problem type
print(f"This is now a Generalized Multiple Allocation {p}-Hub Median Problem.")


This is now a Generalized Multiple Allocation 8-Hub Median Problem.


In [6]:
def select_hubs(probability_vector, p):
    """
    Select p hubs based on the probability vector without replacement and sort them in ascending order.

    Parameters:
        probability_vector (pd.Series): The probability vector with node indices as the index.
        p (int): The number of hubs to select.

    Returns:
        tuple: The selected hubs as both sorted integer indices and original node labels.
    """
    # Select hubs as integer indices
    selected_hubs_indices = np.random.choice(
        range(len(probability_vector)),  # Integer indices
        size=p,                         # Number of hubs to select
        replace=False,                  # Without replacement
        p=probability_vector.values     # Use probability vector for selection
    )

    # Sort indices in ascending order
    selected_hubs_indices = np.sort(selected_hubs_indices)

    # Select hubs as original node labels
    selected_hubs_labels = probability_vector.index[selected_hubs_indices]

    # Print both outputs
    print("Selected Hubs as indices (sorted):", selected_hubs_indices)
    print("Selected Hubs as labels (sorted):", selected_hubs_labels)

    return selected_hubs_indices, selected_hubs_labels

# Example call
# p = 8  # Number of hubs to select
selected_hubs_indices, selected_hubs_labels = select_hubs(probability_vector, p)

Selected Hubs as indices (sorted): [ 5  6 10 12 13 20 21 24]
Selected Hubs as labels (sorted): Index([5, 6, 10, 12, 13, 20, 21, 24], dtype='int64')


In [7]:
def initialize_y_matrix_as_df(n, p, selected_hubs):
    """
    Initialize a y matrix as a Pandas DataFrame where:
    - y[i, i] = 1 if node i is in selected_hubs.
    - y[i, j] = 0 for all other hubs when node i is in selected_hubs.
    - All other entries are set to 1.

    Parameters:
        n (int): Number of nodes.
        p (int): Number of hubs.
        selected_hubs (list): List of selected hubs.

    Returns:
        pd.DataFrame: An nxp DataFrame initialized with the above rules.
    """
    # Initialize the y matrix with ones
    y_matrix = np.ones((n, p), dtype=int)

    # Create a DataFrame with node indices as rows and hub indices as columns
    df_y_matrix = pd.DataFrame(
        y_matrix,
        index=[i for i in range(n)],  # Integer row names (0, 1, 2, ...)
        columns=[hub for hub in selected_hubs]  # Integer column names (hub indices)
    )

    # Set y[i, j] = 0 for j != i if node i is in selected hubs
    for i in selected_hubs:
        df_y_matrix.loc[i, :] = 0
        df_y_matrix.loc[i, i] = 1  # Ensure y[i, i] = 1 for the hub itself

    return df_y_matrix
# Example usage
n = len(sum_W_series)  # Number of nodes
print(type(selected_hubs_indices))
y_df = initialize_y_matrix_as_df(n, p, selected_hubs_indices)

# Display the initialized y matrix as a DataFrame
print("Initialized y matrix as DataFrame:")
display(y_df)

<class 'numpy.ndarray'>
Initialized y matrix as DataFrame:


,5,6,10,12,13,20,21,24
0,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1
5,1,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1


In [8]:
# Initialize the discounting factor
alpha = 0.6

# Print the initialized value
print("Discounting factor alpha:", alpha)

Discounting factor alpha: 0.6


In [9]:
def calculate_cost(i, j, hub_i, hub_j, df_distance, alpha, w_ij):
    """
    Calculate the total cost of traveling from node i to node j via their assigned hubs, multiplied by weight w_ij.

    Parameters:
        i (int): Index of the starting node.
        j (int): Index of the ending node.
        hub_i (int): Hub assigned to the starting node.
        hub_j (int): Hub assigned to the ending node.
        df_distance (pd.DataFrame): DataFrame containing pairwise distances between nodes.
        alpha (float): Discount factor for the distance between hubs.
        w_ij (float): Weight (flow/demand) between node i and node j.

    Returns:
        float: Total weighted cost of traveling from node i to node j.
    """
    # Calculate the distances based on the given formula
    distance_cost = (
        df_distance.loc[i, hub_i] +            # Distance from node i to its hub
        alpha * df_distance.loc[hub_i, hub_j] +  # Discounted distance between hubs
        df_distance.loc[hub_j, j]             # Distance from hub of node j to node j
    )
    total_cost = w_ij * distance_cost  # Multiply by weight
    return total_cost
i, j = 1, 4  # Nodes
hub_i, hub_j = 2, 3  # Hubs assigned
# Extract w_ij from df_W
w_ij = df_W.loc[i, j]
# Calculate the cost
total_weighted_cost = calculate_cost(i, j, hub_i, hub_j, df_distance, alpha, w_ij)

# Expected cost calculation for verification
expected_cost = (
    df_distance.loc[i, hub_i] +
    alpha * df_distance.loc[hub_i, hub_j] +
    df_distance.loc[hub_j, j]
) * w_ij

# Assert to check if the calculated cost matches the expected cost
assert np.isclose(total_weighted_cost, expected_cost), (
    f"Test Failed: Expected {expected_cost}, but got {total_weighted_cost}"
)

# If the assertion passes, print success message in green color
print("\033[92mTest Passed: Total weighted cost from Node 1 to Node 4 via hubs 2 and 3 is correct.\033[0m")


Test Passed: Total weighted cost from Node 1 to Node 4 via hubs 2 and 3 is correct.


In [10]:
def shortest_path(i, j, selected_hubs_indices, df_distance, alpha, df_W, y_df):
    """
    Finds the shortest path from node i to node j using the clever assignment method,
    considering the constraints from y_df.

    Args:
        i: Starting node index.
        j: Ending node index.
        selected_hubs_indices: List of selected hub indices.
        df_distance: Distance DataFrame.
        alpha: Discounting factor.
        df_W: Flow DataFrame.
        y_df: Assignment DataFrame indicating whether a node can use a hub (1 if yes, 0 otherwise).

    Returns:
        A tuple containing:
            - The shortest path cost.
            - The hub assigned to node i.
            - The hub assigned to node j.
    """
    #display(y_df)
    min_cost = float('inf')
    best_hub_i = None
    best_hub_j = None
    for hub_i in selected_hubs_indices:
        for hub_j in selected_hubs_indices:
            # Check if node i can use hub_i and node j can use hub_j
            if y_df.loc[i, hub_i] == 1 and y_df.loc[j, hub_j] == 1:
                cost = calculate_cost(i, j, hub_i, hub_j, df_distance, alpha, df_W.loc[i, j])
                if cost < min_cost:
                    min_cost = cost
                    best_hub_i = hub_i
                    best_hub_j = hub_j
    return min_cost, best_hub_i, best_hub_j
### Test
# Ensure selected_hubs_indices is a list
selected_hubs_indices = list(selected_hubs_indices)

# Initialize a matrix to represent connections
connections_matrix = np.random.randint(0, 2, (n, len(selected_hubs_indices)))  # Random 0s and 1s

# Ensure hubs are assigned to themselves and each node has at least one hub connection
for i in range(n):
    if i in selected_hubs_indices:
        # Get the column index of the hub in selected_hubs_indices
        hub_column = selected_hubs_indices.index(i)
        connections_matrix[i, hub_column] = 1  # Ensure hubs are assigned to themselves
    if not connections_matrix[i].any():  # Ensure at least one connection for each node
        assigned_hub = np.random.choice(range(len(selected_hubs_indices)))
        connections_matrix[i, assigned_hub] = 1

# Convert connections_matrix to y_df_moc
y_df_moc = pd.DataFrame(connections_matrix, index=range(n), columns=selected_hubs_indices)

# Display the initialized y_df_moc
#print("y_df_moc:")
#print(y_df_moc)
#display(y_df_moc)
# Example Usage (Assuming you have the necessary variables defined)
i = 3  # Example starting node
j = 14  # Example ending node
shortest_path_cost, hub_i, hub_j = shortest_path(i, j, selected_hubs_indices, df_distance, alpha, df_W, y_df_moc)

# Verification output
if hub_i is not None and hub_j is not None:
    print(f"\033[92mShortest path cost from node {i} to node {j}: {shortest_path_cost}, node {i} uses hub {hub_i}, node {j} uses hub {hub_j}\033[0m")
else:
    print(f"\033[91mNo valid path found from node {i} to node {j} based on the assignment constraints.\033[0m")


Shortest path cost from node 3 to node 14: 5943.792476732499, node 3 uses hub 10, node 14 uses hub 12


In [11]:
# prompt: calculate an assignment cost from F_assignment_matrix, assignment of i to hub of k is F_assignment_matrix_i,k*df_y_i,k

def calculate_assignment_cost(F_assignment_matrix, df_y, selected_hubs_indices):
    """Calculates the total assignment cost based on the provided matrices.

    Args:
        F_assignment_matrix (np.ndarray or pd.DataFrame): The assignment cost matrix.
        df_y (pd.DataFrame): The assignment matrix indicating node-hub connections.
        selected_hubs_indices (list): A list of selected hub indices.

    Returns:
        float: The total assignment cost.
    """
    total_cost = 0
    n = len(F_assignment_matrix)  # Assuming square matrix
    for i in range(n):
        for k in selected_hubs_indices:
            # Access the cost from the F_assignment_matrix based on index
            total_cost += F_assignment_matrix[i,k] * df_y.iloc[i,selected_hubs_indices.index(k)]
    return total_cost

# Example usage (replace with your actual data)
assignment_cost = calculate_assignment_cost(F_assignment_matrix, y_df_moc, selected_hubs_indices)
print(f"Total Assignment Cost: {assignment_cost}")

Total Assignment Cost: 21984635


In [12]:
# prompt: calculate hub fixed cost from df_nodes F_hub column. It emerges when node i is a hub point

def calculate_hub_fixed_cost(df_nodes, selected_hubs_indices):
    """Calculates the total fixed cost of establishing hubs.

    Args:
        df_nodes (pd.DataFrame): DataFrame containing node information, including F_hub column.
        selected_hubs_indices (list): List of indices of selected hubs.

    Returns:
        float: The total fixed cost of the selected hubs.
    """
    total_fixed_cost = 0
    for hub_index in selected_hubs_indices:
        total_fixed_cost += df_nodes['F_hub'][hub_index]
    return total_fixed_cost

# Example usage (assuming df_nodes and selected_hubs_indices are defined)
hub_fixed_cost = calculate_hub_fixed_cost(df_nodes, selected_hubs_indices)
print(f"Total Hub Fixed Cost: {hub_fixed_cost}")

Total Hub Fixed Cost: 4477153


In [13]:
def check_feasibility(df_y):
    """
    Checks if the feasibility condition is met:
    - The sum of each row in df_y must be >= 1.

    Parameters:
        df_y (pd.DataFrame): The assignment DataFrame where each row represents a node's connection to hubs.

    Returns:
        bool: True if the feasibility condition is met, False otherwise.
    """
    # Check if all rows have a sum >= 1
    feasible = (df_y.sum(axis=1) >= 1).all()
    return feasible
# Example usage
is_feasible = check_feasibility(y_df_moc)
if is_feasible:
    print("\033[92mFeasibility check passed: All rows in df_y have a sum >= 1.\033[0m")
else:
    print("\033[91mFeasibility check failed: Some rows in df_y have a sum < 1.\033[0m")


Feasibility check passed: All rows in df_y have a sum >= 1.


In [14]:
def calculate_total_transportation_cost_with_paths(selected_hubs_indices, df_distance, alpha, df_W, y_df):
    """
    Calculates the total transportation cost by summing min_cost for all node pairs (i, j)
    and extracts all (i->k->l->j) paths as a list.

    Parameters:
        selected_hubs_indices (list): List of selected hub indices.
        df_distance (pd.DataFrame): Distance DataFrame.
        alpha (float): Discounting factor.
        df_W (pd.DataFrame): Flow DataFrame.
        y_df (pd.DataFrame): Assignment DataFrame.

    Returns:
        tuple:
            - float: Total transportation cost for all node pairs.
            - list: List of paths as tuples (node_i, hub_i, hub_j, node_j).
    """
    total_cost = 0
    paths = []

    for i in range(len(df_distance)):
        for j in range(len(df_distance)):
            if i != j:  # Avoid self-loops
                min_cost, hub_i, hub_j = shortest_path(i, j, selected_hubs_indices, df_distance, alpha, df_W, y_df)
                total_cost += min_cost
                paths.append((i, hub_i, hub_j, j))  # Store the path as a tuple

    return total_cost, paths
total_transportation_cost, paths = calculate_total_transportation_cost_with_paths(
    selected_hubs_indices, df_distance, alpha, df_W, y_df
)

print(f"Total Transportation Cost: {total_transportation_cost}")
#print("Paths (i -> k -> l -> j):")
#for path in paths:
#    print(path)

Total Transportation Cost: 3295651.5489473883


In [15]:
def calculate_total_cost(F_assignment_matrix, y_df, selected_hubs_indices, df_nodes, df_distance, alpha, df_W):
    """
    Calculates the total cost including assignment cost, hub fixed cost, and transportation cost.

    Parameters:
        F_assignment_matrix (pd.DataFrame): Assignment cost matrix.
        y_df (pd.DataFrame): Assignment matrix.
        selected_hubs_indices (list): List of selected hub indices.
        df_nodes (pd.DataFrame): DataFrame containing node information.
        df_distance (pd.DataFrame): Distance matrix.
        alpha (float): Discount factor for transportation costs.
        df_W (pd.DataFrame): Flow matrix.

    Returns:
        dict: A dictionary containing the total costs and paths.
    """
    # Calculate assignment cost
    assignment_cost = calculate_assignment_cost(F_assignment_matrix, y_df, selected_hubs_indices)

    # Calculate hub fixed cost
    hub_fixed_cost = calculate_hub_fixed_cost(df_nodes, selected_hubs_indices)

    # Calculate transportation cost and paths
    total_transportation_cost, paths = calculate_total_transportation_cost_with_paths(
        selected_hubs_indices, df_distance, alpha, df_W, y_df
    )

    # Calculate total cost
    total_cost = total_transportation_cost + assignment_cost + hub_fixed_cost

    # Return all results as a dictionary
    return {
        "assignment_cost": assignment_cost,
        "hub_fixed_cost": hub_fixed_cost,
        "transportation_cost": total_transportation_cost,
        "total_cost": total_cost,
        "paths": paths
    }

results = calculate_total_cost(
    F_assignment_matrix, y_df, selected_hubs_indices, df_nodes, df_distance, alpha, df_W
)

print(f"Assignment Cost: {results['assignment_cost']}")
print(f"Hub Fixed Cost: {results['hub_fixed_cost']}")
print(f"Transportation Cost: {results['transportation_cost']}")
print(f"Total Cost: {results['total_cost']}")
paths1 = results['paths']

Assignment Cost: 30374906
Hub Fixed Cost: 4477153
Transportation Cost: 3295651.5489473883
Total Cost: 38147710.54894739


In [16]:

# Assuming 'paths' is the list of tuples (i, k, l, j) from the previous code
# and you want to count the distinct values of 'k' (hub_i) for each 'i' (node_i).

def count_distinct_hubs_per_node(paths):
    """Counts the number of different hubs used by each node as origin.
    Args:
      paths: A list of tuples (i, k, l, j).

    Returns:
      A pandas Series where the index is the node 'i' and the values are the
      number of different hubs used by that node.
    """

    node_hub_counts = {}
    for i, k, _, _ in paths:
        if i not in node_hub_counts:
            node_hub_counts[i] = set()
        node_hub_counts[i].add(k)

    return pd.Series({node: len(hubs) for node, hubs in node_hub_counts.items()})

# Example usage (replace 'paths' with the actual paths from your previous output)
# paths = [(0, 2, 3, 1), (0, 2, 5, 3), (1, 2, 3, 0), ...] # Example

#distinct_hub_counts = count_distinct_hubs_per_node(results['paths'])
#distinct_hub_counts

In [17]:
def update_y_df_based_on_paths(paths, y_df):
    """
    Updates the y_df DataFrame to set df_y[i, k] = 0 if there is no (i, k) pair in paths.

    Parameters:
        paths (list): A list of tuples (i, k, l, j) representing the paths.
        y_df (pd.DataFrame): The assignment DataFrame to be updated.

    Returns:
        pd.DataFrame: Updated y_df DataFrame.
    """
    # Determine all (i, k) pairs from the paths
    used_pairs = {(i, k) for i, k, _, _ in paths}
    print(f"Used Pairs: {sorted(used_pairs)}")
    # Create a copy of y_df to avoid modifying the original directly
    updated_y_df = y_df.copy()

    # Update y_df: set df_y[i, k] = 0 if (i, k) is not in used_pairs
    for i in updated_y_df.index:
        for k in updated_y_df.columns:
            if (i, k) not in used_pairs:
                updated_y_df.loc[i, k] = 0

    return updated_y_df, used_pairs
y_df, used_pairs = update_y_df_based_on_paths(results['paths'], y_df)
#display(y_df)

Used Pairs: [(0, 6), (0, 10), (0, 21), (0, 24), (1, 6), (1, 10), (2, 12), (2, 13), (2, 21), (3, 5), (3, 10), (3, 21), (3, 24), (4, 21), (5, 5), (6, 6), (7, 6), (7, 21), (8, 5), (8, 10), (8, 24), (9, 6), (9, 21), (10, 10), (11, 6), (11, 21), (12, 12), (13, 13), (14, 12), (14, 13), (15, 12), (15, 21), (15, 24), (16, 13), (17, 6), (18, 12), (18, 24), (19, 12), (19, 13), (19, 21), (20, 20), (21, 21), (22, 12), (22, 21), (23, 12), (23, 13), (23, 21), (24, 24)]


In [18]:
results = calculate_total_cost(
    F_assignment_matrix, y_df, selected_hubs_indices, df_nodes, df_distance, alpha, df_W
)

print(f"Assignment Cost: {results['assignment_cost']}")
print(f"Hub Fixed Cost: {results['hub_fixed_cost']}")
print(f"Transportation Cost: {results['transportation_cost']}")
print(f"Total Cost: {results['total_cost']}")
paths2 = results['paths']
assert paths1 == paths2, "Paths should be the same"
print("\033[92mTest Passed: Paths should be the same.\033[0m")

Assignment Cost: 9810511
Hub Fixed Cost: 4477153
Transportation Cost: 3295651.5489473883
Total Cost: 17583315.548947386
Test Passed: Paths should be the same.


In [19]:
def evaluate_cost_improvement(used_pairs, results, F_assignment_matrix, y_df, selected_hubs_indices, df_distance, alpha, df_W):
    """
    Evaluates cost improvement for all used pairs by temporarily updating y_df and recalculating total cost.

    Parameters:
        used_pairs (list): List of (i, k) pairs indicating node-to-hub assignments.
        results (dict): Dictionary containing previous results including total cost.
        F_assignment_matrix (pd.DataFrame): Assignment cost matrix.
        y_df (pd.DataFrame): Assignment matrix.
        selected_hubs_indices (list): List of selected hub indices.
        df_distance (pd.DataFrame): Distance matrix.
        alpha (float): Discount factor for transportation costs.
        df_W (pd.DataFrame): Flow matrix.

    Returns:
        dict: A dictionary with (i, k) pairs as keys and their respective improvements as values (sorted by values in descending order).
    """
    improvement_vector = {}
    print(sorted(used_pairs))

    for i, k in sorted(used_pairs):  # Iterate over sorted used pairs
        # Work with a copy of y_df to ensure original is not modified
        temp_y_df = y_df.copy(deep=True)
        temp_y_df_2 = y_df.copy(deep=True)

        try:
            # Update temp_y_df for the current pair (i, k)
            temp_y_df.loc[i, k] = 0
        except (IndexError, KeyError) as e:
            print(f"Error: {e}")
            pass
        else:
            if check_feasibility(temp_y_df):
                # Recalculate total cost
                total_cost = results['total_cost'] - F_assignment_matrix[i, k]
                sum_shortest_paths = 0
                for j in range(len(df_distance)):
                    shortest_path_cost, _, _ = shortest_path(i, j, selected_hubs_indices, df_distance, alpha, df_W, temp_y_df)
                    prev_shortest_path_cost, _, _ = shortest_path(i, j, selected_hubs_indices, df_distance, alpha, df_W, temp_y_df_2)
                    sum_shortest_paths += shortest_path_cost  - prev_shortest_path_cost
                total_cost += sum_shortest_paths * 2

                # Calculate improvement
                improvement = -(total_cost - results['total_cost'])
                improvement_vector[(i, k)] = improvement

    return dict(sorted(improvement_vector.items(), key=lambda item: item[1], reverse=True))


improvements = evaluate_cost_improvement(
    used_pairs,
    results=results,
    F_assignment_matrix=F_assignment_matrix,
    y_df=y_df,
    selected_hubs_indices=selected_hubs_indices,
    df_distance=df_distance,
    alpha=alpha,
    df_W=df_W
)
print(used_pairs)
print(results)
print(y_df)
print("Sorted Improvement Vector:")
print(improvements)


[(0, 6), (0, 10), (0, 21), (0, 24), (1, 6), (1, 10), (2, 12), (2, 13), (2, 21), (3, 5), (3, 10), (3, 21), (3, 24), (4, 21), (5, 5), (6, 6), (7, 6), (7, 21), (8, 5), (8, 10), (8, 24), (9, 6), (9, 21), (10, 10), (11, 6), (11, 21), (12, 12), (13, 13), (14, 12), (14, 13), (15, 12), (15, 21), (15, 24), (16, 13), (17, 6), (18, 12), (18, 24), (19, 12), (19, 13), (19, 21), (20, 20), (21, 21), (22, 12), (22, 21), (23, 12), (23, 13), (23, 21), (24, 24)]
{(15, 21), (15, 24), (23, 13), (3, 10), (20, 20), (4, 21), (14, 13), (17, 6), (19, 12), (1, 6), (19, 21), (8, 24), (16, 13), (12, 12), (21, 21), (23, 12), (14, 12), (23, 21), (3, 21), (8, 5), (3, 24), (0, 10), (2, 13), (13, 13), (7, 6), (18, 12), (7, 21), (18, 24), (3, 5), (22, 12), (5, 5), (22, 21), (15, 12), (9, 6), (8, 10), (0, 6), (9, 21), (10, 10), (11, 6), (19, 13), (0, 21), (1, 10), (2, 12), (6, 6), (0, 24), (2, 21), (11, 21), (24, 24)}
{'assignment_cost': 9810511, 'hub_fixed_cost': 4477153, 'transportation_cost': 3295651.5489473883, 'tota

In [42]:
import random

def applySA(y_df, results, best_cost, solutions, T):
    """
    Applies a SA by modifying y_df:
    - Selects a random `y_df[i, k] = 1`.
    - Sets it to 0.
    - Changes a randomly selected `y_df[i, k'] = 0` to 1 for the same row `i`.

    Parameters:
        y_df (pd.DataFrame): The current assignment matrix.
        results (dict): The current results containing cost information.
        best_cost (list): A list containing the best cost achieved so far.

    Returns:
        tuple: Updated (y_df, results, best_cost).
    """
    # Create a deep copy of y_df to avoid modifying the original during the process
    y_df_copy = y_df.copy(deep=True)

    # Select a random (i, k) where y_df[i, k] = 1
    current_indices = [(i, k) for i in y_df_copy.index for k in y_df_copy.columns if y_df_copy.loc[i, k] == 1]
    if not current_indices:
        # If no valid indices are found, return the original inputs
        print("No valid (i, k) indices with y_df[i, k] = 1 found.")
        return y_df, results, best_cost, solutions

    i, k = random.choice(current_indices)

    # Select a random (i, k') where y_df[i, k'] = 0
    potential_indices = [k_prime for k_prime in y_df_copy.columns if y_df_copy.loc[i, k_prime] == 0]

    # Set y_df[i, k] = 0
    y_df_copy.loc[i, k] = 0

    if not potential_indices:
        # If no valid indices are found, revert the change and return the original inputs
        y_df_copy.loc[i, k] = 1
        print("No valid (i, k') indices with y_df[i, k'] = 0 found.")
        return y_df, results, best_cost, solutions

    k_prime = random.choice(potential_indices)

    # Set y_df[i, k'] = 1
    y_df_copy.loc[i, k_prime] = 1

    # Calculate the new cost (example placeholder logic, replace with actual cost calculation)
    new_results = calculate_total_cost(F_assignment_matrix, y_df_copy, list(selected_hubs_indices), df_nodes, df_distance, alpha, df_W)
    new_cost = new_results['total_cost']
    # Update best cost if the new cost is better
    if new_cost <= best_cost[-1]:
        best_cost.append(new_cost)
        results = new_results
        solutions.append(results)
        print(f"New best cost: {new_cost}")
        return y_df, results, best_cost, solutions
    elif new_cost <= results['total_cost']:
        # Accept the new solution
        results = new_results
        y_df = y_df_copy
        print(f"Local improvement: {new_cost}")
        return y_df, results, best_cost, solutions
    else:
        p = random.random()
        if p < np.exp(-(new_cost - results['total_cost']) / T):
            # Accept the new solution
            results = new_results
            y_df = y_df_copy
            print(f"Worse Solution is accepted: {new_cost}")
            return y_df, results, best_cost, solutions
        return y_df, results, best_cost, solutions

def applySAIteratively(y_df, results, best_cost, solutions, best_solution, STOP1=50, STOP2=150, T_0 = 50000, beta=0.99, iterations=500):
    """
    Applies SA iteratively for a given number of iterations.

    Parameters:
        y_df (pd.DataFrame): The current assignment matrix.
        results (dict): The current results containing cost information.
        best_cost (list): A list containing the best cost achieved so far.
        iterations (int): Number of iterations to perform local search.

    Returns:
        tuple: Updated (y_df, results, best_cost).
    """
    T = T_0
    best_solution_track = 1
    update = False
    for iter in range(iterations):
        # Perform one iteration of local search
        y_df, results, best_cost, solutions= applySA(y_df, results, best_cost, solutions, T)
        T *= beta
        # Optionally, track progress
        print(f"Iteration {iter + 1}/{iterations} - Best Cost: {best_cost[-1]}")
        if best_cost[-1] < best_solution:
            best_solution = best_cost[-1]
            best_solution_results = results
            best_solution_track = 1
            print(f"New best solution: {best_solution}")
            update = True
            print(f"Update = {update}")
        else:
            best_solution_track += 1
        if iter == STOP1 and update == False:
            print(f"Update = {update}")
            break
        if best_solution_track==STOP2:
            print(f"No Update for {best_solution_track} iterations")
            break

    return y_df, results, best_cost, solutions, best_solution

In [44]:
from tqdm import tqdm
best_cost = []
iteration_number = 100
solutions = []
best_solution = float('inf')
inefficient_iter = 0
for iteration in tqdm(range(iteration_number), desc="Progress", unit="iteration"):
  improvement_value = 0
  cost_reduction_track = [results['total_cost']]  # Start with the initial total cost
  selected_hubs_indices, _ = select_hubs(probability_vector, p)
  y_df = initialize_y_matrix_as_df(n, p, selected_hubs_indices)
  results = calculate_total_cost(
      F_assignment_matrix, y_df, list(selected_hubs_indices), df_nodes, df_distance, alpha, df_W
      )
  y_df, used_pairs = update_y_df_based_on_paths(results['paths'], y_df)
  while improvement_value >= 0:
      #print(f"this is used pairs: {used_pairs}")
      improvements = evaluate_cost_improvement(
          used_pairs,
          results=results,
          F_assignment_matrix=F_assignment_matrix,
          y_df=y_df,
          selected_hubs_indices=selected_hubs_indices,
          df_distance=df_distance,
          alpha=alpha,
          df_W=df_W
          )
      #print(improvements)
      #print(f"this is improvements: {improvements}")
      if len(improvements)==0:
          break
      (i, k), improvement_value = list(improvements.items())[0]
      if improvement_value >= 0:
          # Update total cost
          y_df.loc[i, k] = 0
          results = calculate_total_cost(
              F_assignment_matrix, y_df, list(selected_hubs_indices), df_nodes,
              df_distance, alpha, df_W)
          used_pairs = [pair for pair in used_pairs if pair != (i, k)]
  prev_result = results['total_cost']
  results = calculate_total_cost(
      F_assignment_matrix, y_df, list(selected_hubs_indices), df_nodes, df_distance, alpha, df_W
      )
  print(f"prev_result: {prev_result}, new_result: {results['total_cost']}")
  assert prev_result == results['total_cost'], "Total cost should be the same"
  print("\033[92mTest Passed: Total cost should be the same.\033[0m")
  solutions.append(results)
  best_cost.append(results['total_cost'])
  old_best_solution = best_solution
  y_df, results, solutions, best_cost, best_solution = applySAIteratively(y_df, results, best_cost, solutions, best_solution, iterations=500)
  if old_best_solution == best_solution:
      inefficient_iter += 1
  else:
      inefficient_iter = 0
  if inefficient_iter == 15:
      break
print("Sorted Improvement Vector:")
print(improvements)
print(results['total_cost'])
print(results['assignment_cost'])
print(best_cost)

Progress:   0%|          | 0/100 [00:00<?, ?iteration/s]

Selected Hubs as indices (sorted): [ 4  6  8 14 15 16 20 23]
Selected Hubs as labels (sorted): Index([4, 6, 8, 14, 15, 16, 20, 23], dtype='int64')
Used Pairs: [(0, 8), (0, 15), (1, 6), (2, 23), (3, 8), (4, 4), (5, 8), (5, 14), (5, 15), (5, 16), (5, 20), (6, 6), (7, 4), (7, 6), (8, 8), (9, 4), (9, 6), (9, 15), (10, 6), (10, 8), (10, 15), (11, 4), (11, 6), (11, 23), (12, 14), (12, 15), (12, 16), (12, 20), (13, 16), (13, 20), (14, 14), (15, 15), (16, 16), (17, 6), (18, 14), (18, 15), (18, 16), (18, 20), (19, 4), (19, 23), (20, 20), (21, 4), (22, 4), (22, 14), (22, 23), (23, 23), (24, 14), (24, 15), (24, 16), (24, 20)]
[(0, 8), (0, 15), (1, 6), (2, 23), (3, 8), (4, 4), (5, 8), (5, 14), (5, 15), (5, 16), (5, 20), (6, 6), (7, 4), (7, 6), (8, 8), (9, 4), (9, 6), (9, 15), (10, 6), (10, 8), (10, 15), (11, 4), (11, 6), (11, 23), (12, 14), (12, 15), (12, 16), (12, 20), (13, 16), (13, 20), (14, 14), (15, 15), (16, 16), (17, 6), (18, 14), (18, 15), (18, 16), (18, 20), (19, 4), (19, 23), (20, 20), (

Progress:   1%|          | 1/100 [02:19<3:50:24, 139.64s/iteration]

Iteration 180/500 - Best Cost: 12771977.256005898
No Update for 150 iterations
Selected Hubs as indices (sorted): [ 2  7 16 18 19 21 22 24]
Selected Hubs as labels (sorted): Index([2, 7, 16, 18, 19, 21, 22, 24], dtype='int64')
Used Pairs: [(0, 21), (0, 24), (1, 7), (1, 21), (2, 2), (3, 7), (3, 21), (3, 24), (4, 21), (5, 16), (5, 24), (6, 7), (6, 21), (7, 7), (8, 7), (8, 21), (8, 24), (9, 7), (9, 21), (10, 7), (10, 21), (10, 24), (11, 7), (11, 22), (12, 16), (12, 18), (13, 16), (13, 18), (13, 24), (14, 2), (14, 16), (14, 18), (15, 18), (15, 21), (15, 24), (16, 16), (17, 7), (17, 21), (18, 18), (19, 19), (20, 16), (20, 18), (20, 24), (21, 21), (22, 22), (23, 2), (24, 24)]
[(0, 21), (0, 24), (1, 7), (1, 21), (2, 2), (3, 7), (3, 21), (3, 24), (4, 21), (5, 16), (5, 24), (6, 7), (6, 21), (7, 7), (8, 7), (8, 21), (8, 24), (9, 7), (9, 21), (10, 7), (10, 21), (10, 24), (11, 7), (11, 22), (12, 16), (12, 18), (13, 16), (13, 18), (13, 24), (14, 2), (14, 16), (14, 18), (15, 18), (15, 21), (15, 24),

Progress:   2%|▏         | 2/100 [06:06<5:11:57, 191.00s/iteration]

...

Iteration 51/500 - Best Cost: 12040624.836849365
Update = False
Selected Hubs as indices (sorted): [ 2  8 11 13 17 20 22 24]
Selected Hubs as labels (sorted): Index([2, 8, 11, 13, 17, 20, 22, 24], dtype='int64')
Used Pairs: [(0, 8), (0, 17), (0, 24), (1, 17), (2, 2), (3, 8), (4, 2), (4, 8), (4, 11), (4, 17), (4, 22), (4, 24), (5, 13), (5, 20), (5, 24), (6, 17), (7, 11), (7, 17), (7, 22), (8, 8), (9, 8), (9, 17), (9, 24), (10, 8), (10, 17), (11, 11), (12, 2), (12, 13), (12, 24), (13, 13), (14, 2), (14, 13), (14, 24), (15, 2), (15, 8), (15, 17), (15, 22), (15, 24), (16, 2), (16, 13), (17, 17), (18, 13), (18, 24), (19, 2), (19, 22), (20, 20), (21, 2), (21, 8), (21, 17), (21, 22), (21, 24), (22, 22), (23, 2), (24, 24)]
[(0, 8), (0, 17), (0, 24), (1, 17), (2, 2), (3, 8), (4, 2), (4, 8), (4, 11), (4, 17), (4, 22), (4, 24), (5, 13), (5, 20), (5, 24), (6, 17), (7, 11), (7, 17), (7, 22), (8, 8), (9, 8), (9, 17), (9, 24), (10, 8), (10, 17), (11, 11), (12, 2), (12, 13), (12, 24), (13, 13), (14, 2

Progress:  99%|█████████▉| 99/100 [2:39:06<01:32, 92.67s/iteration]

Iteration 51/500 - Best Cost: 13007261.449298704
Update = False
Selected Hubs as indices (sorted): [ 1  2  5  9 10 11 20 21]
Selected Hubs as labels (sorted): Index([1, 2, 5, 9, 10, 11, 20, 21], dtype='int64')
Used Pairs: [(0, 5), (0, 9), (0, 10), (0, 21), (1, 1), (2, 2), (3, 5), (3, 9), (3, 10), (4, 21), (5, 5), (6, 1), (6, 9), (6, 10), (7, 1), (7, 9), (7, 11), (7, 21), (8, 5), (8, 9), (8, 10), (9, 9), (10, 10), (11, 11), (12, 2), (12, 5), (12, 21), (13, 2), (13, 5), (13, 20), (13, 21), (14, 2), (14, 5), (14, 21), (15, 5), (15, 9), (15, 21), (16, 2), (16, 5), (16, 20), (16, 21), (17, 1), (17, 9), (18, 2), (18, 5), (18, 21), (19, 2), (19, 11), (19, 21), (20, 20), (21, 21), (22, 1), (22, 2), (22, 11), (22, 21), (23, 2), (24, 2), (24, 5), (24, 9), (24, 10), (24, 21)]
[(0, 5), (0, 9), (0, 10), (0, 21), (1, 1), (2, 2), (3, 5), (3, 9), (3, 10), (4, 21), (5, 5), (6, 1), (6, 9), (6, 10), (7, 1), (7, 9), (7, 11), (7, 21), (8, 5), (8, 9), (8, 10), (9, 9), (10, 10), (11, 11), (12, 2), (12, 5), (

Progress: 100%|██████████| 100/100 [2:41:00<00:00, 96.60s/iteration]

Iteration 51/500 - Best Cost: 11826418.140806621
Update = False
Sorted Improvement Vector:
{}
11826418.140806621
2293516
[12944615.230554376, 12930684.594139453, 12870873.55738725, 12771977.256005898, 12771977.256005898, {'assignment_cost': 2186352, 'hub_fixed_cost': 6106469, 'transportation_cost': 3550502.0474520465, 'total_cost': 11843323.047452047, 'paths': [(0, 24, 7, 1), (0, 24, 2, 2), (0, 24, 7, 3), (0, 24, 21, 4), (0, 24, 24, 5), (0, 24, 21, 6), (0, 24, 7, 7), (0, 24, 7, 8), (0, 24, 21, 9), (0, 24, 21, 10), (0, 24, 7, 11), (0, 24, 18, 12), (0, 24, 16, 13), (0, 24, 18, 14), (0, 24, 21, 15), (0, 24, 16, 16), (0, 24, 7, 17), (0, 24, 18, 18), (0, 24, 19, 19), (0, 24, 24, 20), (0, 24, 21, 21), (0, 24, 22, 22), (0, 24, 2, 23), (0, 24, 24, 24), (1, 7, 24, 0), (1, 7, 2, 2), (1, 7, 7, 3), (1, 7, 21, 4), (1, 7, 24, 5), (1, 7, 21, 6), (1, 7, 7, 7), (1, 7, 7, 8), (1, 7, 21, 9), (1, 7, 21, 10), (1, 7, 7, 11), (1, 7, 18, 12), (1, 7, 16, 13), (1, 7, 18, 14), (1, 7, 21, 15), (1, 7, 16, 16), (1,

In [22]:
print(best_solution)

10221228.563886622


In [23]:
selected_hubs_indices, _ = select_hubs(probability_vector, p)

Selected Hubs as indices (sorted): [ 4  6  7 15 18 22 23 24]
Selected Hubs as labels (sorted): Index([4, 6, 7, 15, 18, 22, 23, 24], dtype='int64')


In [40]:
best_cost_random = float('inf')
for _ in range(100):
  y_df = initialize_y_matrix_as_df(n, p, selected_hubs_indices)
  for index in y_df.index:
      for column in y_df.columns:
          y_df.loc[index, column] = 1 if index == column else 0
  #print(y_df)
  for index in y_df.index:
      # Reset the entire row to 0
      y_df.loc[index, :] = 0

      # Ensure y_df.loc[index, column] = 1 if index == column
      if index in y_df.columns:
          y_df.loc[index, index] = 1
      else:
          # Randomly select x columns (without replacement) from the rest
          x = random.randint(1, p)  # Random number between 1 and p
          available_columns = [col for col in y_df.columns if col != index]
          random_columns = random.sample(available_columns, x)

          # Set selected columns to 1
          y_df.loc[index, random_columns] = 1
  #print(y_df)
  results = calculate_total_cost(
        F_assignment_matrix, y_df, list(selected_hubs_indices), df_nodes, df_distance, alpha, df_W
        )
  y_df, used_pairs = update_y_df_based_on_paths(results['paths'], y_df)
  results = calculate_total_cost(
        F_assignment_matrix, y_df, list(selected_hubs_indices), df_nodes, df_distance, alpha, df_W
        )
  print(results['total_cost'])
  if best_cost_random > results['total_cost']:
    best_cost_random = results['total_cost']
    print(best_cost_random)

  best_cost_random = results['total_cost']

print(best_cost_random)

    4   6   7   15  18  22  23  24
0    0   0   0   1   0   1   1   0
1    1   1   1   1   1   1   1   1
2    1   1   1   1   1   1   0   1
3    1   0   0   0   0   0   0   1
4    1   0   0   0   0   0   0   0
5    0   1   1   1   1   0   1   0
6    0   1   0   0   0   0   0   0
7    0   0   1   0   0   0   0   0
8    1   1   1   1   1   1   1   0
9    1   1   1   0   1   1   1   1
10   0   1   1   1   1   1   1   0
11   1   1   1   1   1   1   1   1
12   1   0   0   0   0   0   0   0
13   1   1   1   1   0   0   1   1
14   1   1   1   1   1   0   1   1
15   0   0   0   1   0   0   0   0
16   1   0   0   1   0   0   0   1
17   1   1   1   1   1   1   1   1
18   0   0   0   0   1   0   0   0
19   0   1   0   1   1   1   0   1
20   0   0   0   1   0   1   0   0
21   1   1   0   1   1   1   0   1
22   0   0   0   0   0   1   0   0
23   0   0   0   0   0   0   1   0
24   0   0   0   0   0   0   0   1
Used Pairs: [(0, 15), (1, 6), (1, 7), (2, 4), (2, 15), (2, 18), (2, 22), (3, 4), (3, 24), 

In [41]:
print((best_cost_random-best_solution)/best_cost_random)

0.3336751597940797


In [45]:
print(best_solution)

10428835.923729628


In [46]:
print((best_cost_random-best_solution)/best_cost_random)

0.32014117608475123
